<a href="https://colab.research.google.com/github/janvierk/MRP/blob/main/MRP_XLM_ROberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/MRP')

XLM-  ROberta

In [4]:
#installing requitred libries and supressing  the print-out
#!pip install requests==2.31.0 --upgrade

#installing requitred libries and supressing  the print-out
#!pip install requests==2.31.0 --upgrade -q > NUL
!pip install accelerate -U -q > NUL
!pip install transformers -q > NUL
!pip install torch -q > NUL
#!pip install -U transformers -q >NULip install datasets -q > NUL
!pip install tensorflow -q > NUL
!pip install datasets -q > NUL
!pip install evaluate -q > NUL

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers  import Trainer
from transformers import TrainingArguments
from transformers import XLMRobertaTokenizer
from transformers import XLMRobertaForSequenceClassification
import seaborn as sn
import matplotlib
from transformers import AutoTokenizer, AutoModelForTokenClassification
import transformers as tf
import torch
from datasets import load_dataset
import datasets
import  evaluate

In [6]:
#swahili_datasets
swahili_testing = pd.read_csv("swahili_testing.csv")
swahili_train = pd.read_csv("swahili_train.csv")
swahili_validation = pd.read_csv("swahili_validation.csv")

In [7]:
#english_dataset
english_testing = pd.read_csv("english_testing.csv")
english_train = pd.read_csv("english_train.csv")
english_validation = pd.read_csv("english_validation.csv")

In [8]:
tokenizer_xlmro = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

# Tokenize the dataset
def tokenize_xlmr(text, maximum_length =512):
    return tokenizer_xlmro(text,
                     padding="max_length",
                     truncation=True,
                           max_length = maximum_length)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
#tokenize swahili dataset for xlm-roberta
tokenized_swahili_train_xlm =  swahili_train["text"].apply(tokenize_xlmr)
tokenized_swahili_validation_xlm = swahili_validation["text"].apply(tokenize_xlmr)
tokenized_swahili_testing_xlm = swahili_testing["text"].apply(tokenize_xlmr)

In [10]:
#tokenize english dataset for xlm-roberta
tokenized_english_train_xlm =  english_train["text"].apply(tokenize_xlmr)
tokenized_english_validation_xlm = english_validation["text"].apply(tokenize_xlmr)
tokenized_english_testing_xlm = english_testing["text"].apply(tokenize_xlmr)

In [11]:
#checking if the text has been tokenized accordingly
for i in range(3):
    print(f"Tokenized entry {i+1}:")
    print(f"Input IDs: {tokenized_swahili_train_xlm.iloc[i]['input_ids']}")
    print(f"Attention Mask: {tokenized_swahili_train_xlm.iloc[i]['attention_mask']}")
    print("\n")

Tokenized entry 1:
Input IDs: [0, 1519, 78468, 1608, 72275, 200, 150, 85922, 130968, 39, 138667, 85035, 200, 112114, 34590, 11, 200, 301, 49366, 7950, 416, 120512, 76, 24, 884, 3010, 1294, 5538, 198511, 31, 151, 105518, 21, 200359, 7078, 34721, 13019, 15026, 2906, 34028, 18285, 13019, 18, 2037, 634, 5203, 151, 180486, 19562, 35466, 24, 103115, 151, 97188, 151, 141017, 151, 25243, 151, 134657, 5538, 118974, 259, 34236, 4, 2541, 5, 29638, 18856, 259, 1401, 163822, 27610, 4, 22758, 12452, 36512, 4, 259, 210293, 228, 34590, 11, 178235, 97804, 234538, 19562, 760, 228, 134867, 75, 53229, 30714, 259, 17982, 95560, 259, 40891, 26803, 139470, 11877, 34721, 18802, 78439, 14, 1651, 204835, 24, 6275, 14, 48955, 496, 80, 97188, 209, 13019, 1158, 10075, 7230, 40823, 151, 228, 199190, 760, 38765, 5748, 5, 155, 1062, 145334, 24329, 53229, 20385, 234538, 13860, 178235, 39700, 4268, 75, 57268, 2477, 70590, 433, 36662, 24, 75, 48949, 1622, 38765, 370, 760, 5748, 80845, 370, 69732, 68106, 114428, 48955, 2

In [12]:
#getting labels for swahili datasets
swahili_train_labels = swahili_train["label"]
swahili_validation_labels = swahili_validation["label"]
swahili_testing_labels = swahili_testing["label"]

In [13]:
#getting labels for english datasets
english_train_labels = english_train["label"]
english_validation_labels = english_validation["label"]
english_testing_labels = english_testing["label"]

In [14]:
#extracting input ids and attention mask for swahili training
input_ids_swahili_train_xlm= [item["input_ids"] for item in tokenized_swahili_train_xlm]
attention_mask_swahili_train_xlm = [item["attention_mask"] for item in tokenized_swahili_train_xlm]
swahili_encodings_train_xlm = {"input_ids": input_ids_swahili_train_xlm,
             "attention_mask": attention_mask_swahili_train_xlm}

In [15]:
#extracting input ids and attention mask for swahili validation
input_ids_swahili_validation_xlm = [item["input_ids"] for item in tokenized_swahili_validation_xlm]
attention_mask_swahili_validation_xlm = [item["attention_mask"] for item in tokenized_swahili_validation_xlm]
swahili_encodings_validation_xlm = {"input_ids": input_ids_swahili_validation_xlm,
             "attention_mask": attention_mask_swahili_validation_xlm}

In [16]:
#extracting input ids and attention mask for swahili testing
input_ids_swahili_testing_xlm= [item["input_ids"] for item in tokenized_swahili_testing_xlm]
attention_mask_swahili_testing_xlm = [item["attention_mask"] for item in tokenized_swahili_testing_xlm]
swahili_encodings_testing_xlm = {"input_ids": input_ids_swahili_testing_xlm,
             "attention_mask": attention_mask_swahili_testing_xlm}

In [17]:
#extracting input ids and attention mask for english training
input_ids_english_train_xlm= [item["input_ids"] for item in tokenized_english_train_xlm]
attention_mask_english_train_xlm = [item["attention_mask"] for item in tokenized_english_train_xlm]
english_encodings_train_xlm = {"input_ids": input_ids_english_train_xlm,
             "attention_mask": attention_mask_english_train_xlm}

In [18]:
#extracting input ids and attention mask for english validation
input_ids_english_validation_xlm= [item["input_ids"] for item in tokenized_english_validation_xlm]
attention_mask_english_validation_xlm = [item["attention_mask"] for item in tokenized_english_validation_xlm]
english_encodings_validation_xlm = {"input_ids": input_ids_english_validation_xlm,
             "attention_mask": attention_mask_english_validation_xlm}

In [19]:
#extracting input ids and attention mask for english testing
input_ids_english_testing_xlm= [item["input_ids"] for item in tokenized_english_testing_xlm]
attention_mask_english_testing_xlm = [item["attention_mask"] for item in tokenized_english_testing_xlm]
english_encodings_testing_xlm = {"input_ids": input_ids_english_testing_xlm,
             "attention_mask": attention_mask_english_testing_xlm}

In [20]:
#creating tensor data to feed into the model
import torch
from torch.utils.data import Dataset, DataLoader

class TextDataset (Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item["labels"] = torch.tensor(self.labels[idx])
    return item
  def __len__(self):
    return len(self.labels)

In [21]:
#swahili ready data
swahili_ready_train_xlm = TextDataset(swahili_encodings_train_xlm, swahili_train_labels)
swahili_ready_validation_xlm = TextDataset(swahili_encodings_validation_xlm, swahili_validation_labels)
swahili_ready_testing_xlm = TextDataset(swahili_encodings_testing_xlm, swahili_testing_labels)

In [22]:
#english ready data
english_ready_train_xlm = TextDataset(english_encodings_train_xlm, english_train_labels)
english_ready_validation_xlm = TextDataset(english_encodings_validation_xlm, english_validation_labels)
english_ready_testing_xlm = TextDataset(english_encodings_testing_xlm, english_testing_labels)

In [23]:
#models
xlm_robert_model_english= XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels = english_train["label"].nunique())
xlm_robert_model_swahili= XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels = swahili_train["label"].nunique())

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#

In [24]:
#training arguments
training_args_swa_xlmr = TrainingArguments(
    output_dir="./swaxlmr_results1",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=200,
    weight_decay=0.01,
    logging_dir="./swaxlmr_logging1",
    logging_steps=20,
    load_best_model_at_end=True,
    eval_strategy="epoch",
    save_strategy="epoch"
)

training_args_eng_xlmr = TrainingArguments(
    output_dir="./engxlmr_results1",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=200,
    weight_decay=0.01,
    logging_dir="./engxlmr_logging1",
    logging_steps=20,
    load_best_model_at_end=True,
    eval_strategy="epoch",
    save_strategy="epoch"
)

In [ ]:
#computing the evaluation metrics
"""
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    accuracy = accuracy_metric.compute(predictions=preds, references=labels)
    f1 = f1_metric.compute(predictions=preds, references=labels, average='weighted')
    precision = precision_metric.compute(predictions=preds, references=labels, average='weighted')
    recall = recall_metric.compute(predictions=preds, references=labels, average='weighted')

    return {
        'accuracy': accuracy['accuracy'],
        'f1': f1['f1'],
        'precision': precision['precision'],
        'recall': recall['recall']
    }"""

In [25]:
from transformers import EarlyStoppingCallback
from sklearn.metrics import confusion_matrix

In [26]:
#computing the evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    accuracy = accuracy_metric.compute(predictions=preds, references=labels)
    f1 = f1_metric.compute(predictions=preds, references=labels, average='weighted')
    precision = precision_metric.compute(predictions=preds, references=labels, average='weighted')
    recall = recall_metric.compute(predictions=preds, references=labels, average='weighted')
    conf_matrix = confusion_matrix(labels, preds)
    conf_matrix_list = conf_matrix.tolist()

    return {
        'accuracy': accuracy['accuracy'],
        'f1': f1['f1'],
        'precision': precision['precision'],
        'recall': recall['recall'],
        'conf_matrix': conf_matrix_list
    }

In [27]:
import torch
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch.nn as nn
# Calculate class weights using sklearn's compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(swahili_train_labels), y=swahili_train_labels)

# Convert class weights to a tensor for PyTorch
class_weights = torch.tensor(class_weights, dtype=torch.float)

In [28]:
#Define your custom Trainer class with class weights
class CustomTrainer(Trainer):
    def __init__(self, class_weights, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights.to(self.args.device)  # Move weights to the appropriate device

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Define the loss function with class weights
        loss_fct = nn.CrossEntropyLoss(weight=self.class_weights)
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss

In [29]:
#defining swahili xlm trainer
xlmr_trainer_swahili= CustomTrainer(
    model=xlm_robert_model_swahili,
    args=training_args_swa_xlmr,
    train_dataset=swahili_ready_train_xlm,
    eval_dataset=swahili_ready_validation_xlm,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)],
    class_weights=class_weights)

    # Use the modified compute_metrics function

In [35]:
#defining swahili xlm trainer
xlmr_trainer_english= CustomTrainer(
    model=xlm_robert_model_english,
    args=training_args_eng_xlmr,
    train_dataset=english_ready_train_xlm,
    eval_dataset=english_ready_validation_xlm,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)],
    class_weights=class_weights)

In [30]:
#defining english xlm trainer
"""



xlmr_trainer_english = Trainer(
    model=xlm_robert_model_english,
    args=training_args_eng_xlmr,
    train_dataset=english_ready_train_xlm,
    eval_dataset=english_ready_validation_xlm,
    compute_metrics=compute_metrics,  # Use the modified compute_metrics function
)"""

In [36]:
#training swahili xlm model
xlmr_trainer_swahili.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Conf Matrix
1,0.434200,0.560057,0.919106,0.916832,0.916936,0.919106,"[[2978, 80, 40, 32, 85], [59, 277, 0, 0, 2], [21, 1, 508, 4, 18], [107, 0, 3, 90, 0], [19, 3, 3, 1, 1578]]"
2,0.278300,0.567659,0.923507,0.922612,0.923270,0.923507,"[[2990, 19, 47, 69, 90], [74, 255, 0, 3, 6], [10, 0, 516, 4, 22], [79, 0, 5, 116, 0], [18, 1, 4, 1, 1580]]"


Trainer is attempting to log a value of "[[2978, 80, 40, 32, 85], [59, 277, 0, 0, 2], [21, 1, 508, 4, 18], [107, 0, 3, 90, 0], [19, 3, 3, 1, 1578]]" of type <class 'list'> for key "eval/conf_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[2990, 19, 47, 69, 90], [74, 255, 0, 3, 6], [10, 0, 516, 4, 22], [79, 0, 5, 116, 0], [18, 1, 4, 1, 1580]]" of type <class 'list'> for key "eval/conf_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=2216, training_loss=0.35944859011078567, metrics={'train_runtime': 937.86, 'train_samples_per_second': 56.701, 'train_steps_per_second': 3.544, 'total_flos': 9328064386633728.0, 'train_loss': 0.35944859011078567, 'epoch': 2.0})

In [37]:
#training english xlm modelS
xlmr_trainer_english.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Conf Matrix
1,0.139000,0.288737,0.956169,0.952403,0.955021,0.956169,"[[3152, 29, 5, 5, 4], [65, 2300, 0, 0, 1], [43, 1, 121, 1, 0], [62, 2, 1, 29, 0], [37, 2, 0, 1, 48]]"
2,0.123200,0.160127,0.965815,0.964829,0.964573,0.965815,"[[3111, 45, 13, 20, 6], [27, 2339, 0, 0, 0], [20, 0, 145, 1, 0], [38, 2, 1, 52, 1], [22, 3, 1, 2, 60]]"
3,0.215800,0.139448,0.964292,0.964593,0.965100,0.964292,"[[3068, 51, 19, 37, 20], [15, 2350, 0, 0, 1], [18, 0, 146, 2, 0], [28, 2, 1, 62, 1], [10, 3, 1, 2, 72]]"


Trainer is attempting to log a value of "[[3152, 29, 5, 5, 4], [65, 2300, 0, 0, 1], [43, 1, 121, 1, 0], [62, 2, 1, 29, 0], [37, 2, 0, 1, 48]]" of type <class 'list'> for key "eval/conf_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[3111, 45, 13, 20, 6], [27, 2339, 0, 0, 0], [20, 0, 145, 1, 0], [38, 2, 1, 52, 1], [22, 3, 1, 2, 60]]" of type <class 'list'> for key "eval/conf_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[3068, 51, 19, 37, 20], [15, 2350, 0, 0, 1], [18, 0, 146, 2, 0], [28, 2, 1, 62, 1], [10, 3, 1, 2, 72]]" of type <class 'list'> for key "eval/conf_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=3324, training_loss=0.14548586218401557, metrics={'train_runtime': 1420.9537, 'train_samples_per_second': 37.424, 'train_steps_per_second': 2.339, 'total_flos': 1.3992096579950592e+16, 'train_loss': 0.14548586218401557, 'epoch': 3.0})

In [38]:
xlmr_test_swahili = xlmr_trainer_swahili.evaluate(eval_dataset=swahili_ready_testing_xlm)

Trainer is attempting to log a value of "[[2973, 72, 17, 42, 66], [65, 311, 1, 0, 2], [18, 0, 554, 9, 11], [111, 0, 4, 80, 0], [11, 3, 8, 0, 1551]]" of type <class 'list'> for key "eval/conf_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


In [39]:
xlmr_test_english = xlmr_trainer_english.evaluate(eval_dataset=english_ready_testing_xlm)

Trainer is attempting to log a value of "[[3044, 65, 15, 37, 30], [23, 2282, 0, 3, 5], [27, 3, 163, 1, 2], [35, 4, 1, 66, 2], [18, 2, 0, 1, 80]]" of type <class 'list'> for key "eval/conf_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


In [40]:
print(xlmr_test_swahili)

{'eval_loss': 0.5445060729980469, 'eval_accuracy': 0.925537315958707, 'eval_f1': 0.9230462809577358, 'eval_precision': 0.9220361717218518, 'eval_recall': 0.925537315958707, 'eval_conf_matrix': [[2973, 72, 17, 42, 66], [65, 311, 1, 0, 2], [18, 0, 554, 9, 11], [111, 0, 4, 80, 0], [11, 3, 8, 0, 1551]], 'eval_runtime': 44.5216, 'eval_samples_per_second': 132.722, 'eval_steps_per_second': 8.311, 'epoch': 2.0}


In [41]:
print(xlmr_test_english)

{'eval_loss': 0.21139897406101227, 'eval_accuracy': 0.953630055847013, 'eval_f1': 0.9537924531052919, 'eval_precision': 0.9543560837932983, 'eval_recall': 0.953630055847013, 'eval_conf_matrix': [[3044, 65, 15, 37, 30], [23, 2282, 0, 3, 5], [27, 3, 163, 1, 2], [35, 4, 1, 66, 2], [18, 2, 0, 1, 80]], 'eval_runtime': 44.4653, 'eval_samples_per_second': 132.89, 'eval_steps_per_second': 8.321, 'epoch': 3.0}
